In [126]:
!pip install tensorflow_recommenders

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs
from typing import List, Union, Dict, Text


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [107]:
movies_df = pd.read_csv('data/movies_cleaned.csv')
users_df = pd.read_csv('data/users_cleaned.csv')
ratings_df = pd.read_csv('data/ratings_cleaned.csv' )

In [108]:
# Merge all data into a single DataFrame
merged_df = pd.merge(ratings_df, users_df, on='userId')
merged_df = pd.merge(merged_df, movies_df, left_on='movieId', right_on='ml_movieId')

merged_df['userId']=merged_df['userId'].astype(str)
merged_df['Title']=merged_df['Title'].astype(str)

# Convert the merged DataFrame to a TensorFlow Dataset
ratings = tf.data.Dataset.from_tensor_slices(dict(merged_df))

In [109]:
# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["Title"],
    "user_id": x["userId"],
    "user_rating": x["rating"],
})
movies = tf.data.Dataset.from_tensor_slices(movies_df["Title"].unique())

In [110]:
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

embedding_dimension = 32

user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

In [111]:
class MovielensModel(tfrs.models.Model):

    def __init__(self, user_model, movie_model, rating_weight: float, retrieval_weight: float) -> None:
        super().__init__()

        self.movie_model: tf.keras.layers.Layer = movie_model
        self.user_model: tf.keras.layers.Layer = user_model

        self.rating_model = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation="relu"),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dense(1),
        ])

        self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()],
        )
        self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=movies.batch(128).map(self.movie_model)
            )
        )

        self.rating_weight = rating_weight
        self.retrieval_weight = retrieval_weight

    def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
        user_embeddings = self.user_model(features["user_id"])
        movie_embeddings = self.movie_model(features["movie_title"])

        return (
            user_embeddings,
            movie_embeddings,
            self.rating_model(
                tf.concat([user_embeddings, movie_embeddings], axis=1)
            ),
        )

    def compute_loss(self, features: Dict[str, tf.Tensor], training=False) -> tf.Tensor:
        ratings = features.pop("user_rating")

        user_embeddings, movie_embeddings, rating_predictions = self(features)

        rating_loss = self.rating_task(
            labels=ratings,
            predictions=rating_predictions,
        )
        retrieval_loss = self.retrieval_task(user_embeddings, movie_embeddings)

        return (self.rating_weight * rating_loss
                + self.retrieval_weight * retrieval_loss)

    def recommend(self, user_id: str, k: int = 5):
        user_id_tensor = tf.convert_to_tensor([user_id])
        user_embedding = self.user_model(user_id_tensor)
        movie_embeddings = self.movie_model.layers[-1].embeddings
        scores = tf.linalg.matmul(user_embedding, movie_embeddings, transpose_b=True)
        top_k_movie_indices = tf.nn.top_k(scores, k=k).indices.numpy()
        return [unique_movie_titles[i] for i in top_k_movie_indices[0]]
    
    

In [127]:
def add_new_user_ratings(user_id: str, watched_movies: List[str], user_ratings: List[Union[int, float]]):
    # Declare global variables at the beginning of the function
    global ratings
    global unique_user_ids

    # Ensure the user is new
    assert user_id not in unique_user_ids, "The new user id already exists in the data."

    # Ensure the movies exist in the dataset
    for movie in watched_movies:
        assert movie in unique_movie_titles, f"The movie {movie} does not exist in the data."

    # Convert user_ratings to integers
    user_ratings = [int(rating) for rating in user_ratings]

    # Create new user data
    new_user_data = pd.DataFrame({
        'user_id': [user_id] * len(watched_movies),
        'movie_title': watched_movies,
        'user_rating': user_ratings,
    })

    # Convert the DataFrame to a TensorFlow Dataset
    new_ratings = tf.data.Dataset.from_tensor_slices(dict(new_user_data))

    # Update the global 'ratings' variable
    ratings = ratings.concatenate(new_ratings)

    # Update the global 'unique_user_ids' variable
    unique_user_ids = np.concatenate([unique_user_ids, np.array([user_id])])

In [128]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [129]:
# Define model checkpoints and early stopping
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="model_checkpoint",
    save_weights_only=True,
    monitor="val_loss",
    mode="min",
    save_best_only=True
)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=3,
    mode="min",
    restore_best_weights=True
)

In [116]:
# Rating-focused model
rating_model = MovielensModel(user_model, movie_model, rating_weight=1.0, retrieval_weight=0.0)
rating_model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
rating_model.fit(
    cached_train,
    epochs=10,
    validation_data=cached_test,
    callbacks=[checkpoint_callback, early_stopping_callback]
)
rating_metrics = rating_model.evaluate(cached_test, return_dict=True)
print(f"Retrieval top-100 accuracy: {rating_metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {rating_metrics['root_mean_squared_error']:.3f}.")

Epoch 1/10
10/10 [==============================] - 36s 3s/step - root_mean_squared_error: 2.3736 - factorized_top_k/top_1_categorical_accuracy: 1.7500e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0013 - factorized_top_k/top_10_categorical_accuracy: 0.0029 - factorized_top_k/top_50_categorical_accuracy: 0.0154 - factorized_top_k/top_100_categorical_accuracy: 0.0309 - loss: 5.1141 - regularization_loss: 0.0000e+00 - total_loss: 5.1141 - val_root_mean_squared_error: 1.0030 - val_factorized_top_k/top_1_categorical_accuracy: 2.0000e-04 - val_factorized_top_k/top_5_categorical_accuracy: 0.0011 - val_factorized_top_k/top_10_categorical_accuracy: 0.0022 - val_factorized_top_k/top_50_categorical_accuracy: 0.0133 - val_factorized_top_k/top_100_categorical_accuracy: 0.0280 - val_loss: 1.0011 - val_regularization_loss: 0.0000e+00 - val_total_loss: 1.0011
Epoch 2/10
10/10 [==============================] - 28s 3s/step - root_mean_squared_error: 0.9877 - factorized_top_k/top_1_categorical_a

In [117]:
# Retrieval-focused model
retrieval_model = MovielensModel(user_model, movie_model, rating_weight=0.0, retrieval_weight=1.0)
retrieval_model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
retrieval_model.fit(
    cached_train,
    epochs=10,
    validation_data=cached_test,
    callbacks=[checkpoint_callback, early_stopping_callback]
)
retrieval_metrics = retrieval_model.evaluate(cached_test, return_dict=True)
print(f"Retrieval top-100 accuracy: {retrieval_metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {retrieval_metrics['root_mean_squared_error']:.3f}.")


Epoch 1/10
10/10 [==============================] - 46s 5s/step - root_mean_squared_error: 4.0551 - factorized_top_k/top_1_categorical_accuracy: 0.0030 - factorized_top_k/top_5_categorical_accuracy: 0.0303 - factorized_top_k/top_10_categorical_accuracy: 0.0673 - factorized_top_k/top_50_categorical_accuracy: 0.2809 - factorized_top_k/top_100_categorical_accuracy: 0.3520 - loss: 70241.6733 - regularization_loss: 0.0000e+00 - total_loss: 70241.6733 - val_root_mean_squared_error: 4.0268 - val_factorized_top_k/top_1_categorical_accuracy: 0.0049 - val_factorized_top_k/top_5_categorical_accuracy: 0.0356 - val_factorized_top_k/top_10_categorical_accuracy: 0.0786 - val_factorized_top_k/top_50_categorical_accuracy: 0.3650 - val_factorized_top_k/top_100_categorical_accuracy: 0.4918 - val_loss: 29471.7871 - val_regularization_loss: 0.0000e+00 - val_total_loss: 29471.7871
Epoch 2/10
10/10 [==============================] - 57s 6s/step - root_mean_squared_error: 4.0462 - factorized_top_k/top_1_categ

In [149]:
# Balanced model
balanced_model = MovielensModel(user_model, movie_model, rating_weight=1.0, retrieval_weight=1.0)
balanced_model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
balanced_model.fit(
    cached_train,
    epochs=10,
    validation_data=cached_test,
    callbacks=[checkpoint_callback, early_stopping_callback]
)
balanced_metrics = balanced_model.evaluate(cached_test, return_dict=True)
print(f"Retrieval top-100 accuracy: {balanced_metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {balanced_metrics['root_mean_squared_error']:.3f}.")


Epoch 1/10
10/10 [==============================] - 46s 4s/step - root_mean_squared_error: 3.8120 - factorized_top_k/top_1_categorical_accuracy: 0.0183 - factorized_top_k/top_5_categorical_accuracy: 0.1573 - factorized_top_k/top_10_categorical_accuracy: 0.2806 - factorized_top_k/top_50_categorical_accuracy: 0.7030 - factorized_top_k/top_100_categorical_accuracy: 0.8380 - loss: 65971.2926 - regularization_loss: 0.0000e+00 - total_loss: 65971.2926 - val_root_mean_squared_error: 1.3101 - val_factorized_top_k/top_1_categorical_accuracy: 0.0010 - val_factorized_top_k/top_5_categorical_accuracy: 0.0144 - val_factorized_top_k/top_10_categorical_accuracy: 0.0501 - val_factorized_top_k/top_50_categorical_accuracy: 0.3556 - val_factorized_top_k/top_100_categorical_accuracy: 0.5261 - val_loss: 30644.0430 - val_regularization_loss: 0.0000e+00 - val_total_loss: 30644.0430
Epoch 2/10
10/10 [==============================] - 57s 6s/step - root_mean_squared_error: 1.1885 - factorized_top_k/top_1_categ

In [145]:
# Add a new user's ratings
add_new_user_ratings(
    user_id="new_user_3",
    watched_movies=[
        unique_movie_titles[12],
        unique_movie_titles[23],
       unique_movie_titles[99]
    ],
    user_ratings=[4.0, 3.5, 5.0]
)


In [154]:
def get_unique_movie_title(movie_title):
    for index, unique_title in enumerate(unique_movie_titles):
        # Convert unique_title to string and remove the 'b' prefix and single quotes
        unique_title_str = str(unique_title)[2:-1]
        if movie_title == unique_title_str:
            return index
    return None


In [155]:
movie_title = '8 Seconds'
unique_title_index = get_unique_movie_title(movie_title)
if unique_title_index is not None:
    print(f"The corresponding unique title index for '{movie_title}' is {unique_title_index}.")
else:
    print(f"No matching unique title found for '{movie_title}'.")


The corresponding unique title index for '8 Seconds' is 20.


In [148]:
# Replace "42" and "Dances with Wolves (1990)" with valid user_id and movie_title from your data
user_id = "new_user_3"  # Replace with a valid user_id
recommendations = balanced_model.recommend(user_id, k=5)
print(f"Top 5 movie recommendations for user {user_id}: {recommendations}")


Top 5 movie recommendations for user 43: [b'Shall We Dance?', b'Raining Stones', b'Toys', b'A Century of Cinema', b'American Buffalo']
